In [2]:
import numpy as np
import pandas as pd

processed_dir = "../datasets/processed/new_pts/"

ppi_dataset = pd.read_csv("../datasets/processed/ppi_dataset.csv")
npy_ar = np.array(ppi_dataset)
npy_ar

array([['NP_604391', 'P16220', '2LXT', ..., 'P51532', '2GRC', 1],
       ['NP_000263', 'O15259', '1S1N', ..., 'P21333', '2AAV', 1],
       ['NP_057424', 'Q8TEU7', '2D93', ..., 'P10114', '1KAO', 1],
       ...,
       ['NP_006182', 'Q9UQ80', '2Q8K', ..., 'P21860', '1M6B', 1],
       ['NP_003205', 'Q99594', '5EMW', ..., 'P48147', '3DDU', 0],
       ['NP_001955', 'P18146', '4R2A', ..., 'P53396', '3MWD', 0]],
      dtype=object)

In [57]:
import torch
from torch_geometric.data import Dataset as Dataset_TG
import os


class LabelledDataset(Dataset_TG):
    def __init__(self, npy_file, processed_dir):
        self.npy_ar = np.load(npy_file)
        self.processed_dir = processed_dir

        self.protein_1 = self.npy_ar[:, 2]
        self.protein_2 = self.npy_ar[:, 5]

        self.label = self.npy_ar[:, 6].astype(float)
        self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        prot_1_path = os.path.join(self.processed_dir, self.protein_1[index] + ".pt")
        prot_2_path = os.path.join(self.processed_dir, self.protein_2[index] + ".pt")

        print(f"First prot is {prot_1_path}")
        print(f"Second prot is {prot_2_path}")

        prot_1 = torch.load(prot_1_path)
        prot_2 = torch.load(prot_2_path)

        return prot_1, prot_2, torch.tensor(self.label[index])

    def len(self):
        return len(self)

    def get(self, index):
        return self[index]

In [ ]:
dataset = LabelledDataset(npy_ar, processed_dir)

In [61]:
import math

# Make iterables using dataloader class
DATASET_SIZE = len(dataset)
trainset, testset = torch.utils.data.random_split(
    dataset,
    [math.floor(0.8 * DATASET_SIZE), DATASET_SIZE - math.floor(0.8 * DATASET_SIZE)],
)

In [68]:
from torch_geometric.loader import DataLoader

BATCH_SIZE = 4

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

len(trainloader), len(testloader)

(4444, 1111)